In [1]:
#set up smart sensing of os
if(Sys.info()[1]=='Windows'){
  j = "J:/"
} else{
  j = '/home/j/'
}
options(scipen=999)

In [2]:
#load libraries
pack_lib = '/home/j/temp/dccasey/temperature/packages/'
.libPaths(pack_lib)
library('parallel')
library('sp')
library('rgdal', lib.loc = '/home/j/temp/geospatial/packages')
for(ppp in c('raster','ncdf4','data.table')){
  library(ppp, lib.loc = pack_lib, character.only =T)
}

rgdal: version: 1.2-4, (SVN revision 643)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 1.7.3, released 2010/11/10
 Path to GDAL shared files: /usr/share/gdal
 GDAL does not use iconv for recoding strings.
 Loaded PROJ.4 runtime: Rel. 4.7.1, 23 September 2009, [PJ_VERSION: 470]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.2-3 

Attaching package: ‘data.table’

The following object is masked from ‘package:raster’:

    shift



In [21]:
source(paste0(j,'/temp/central_comp/libraries/current/r/get_location_metadata.R'))

In [3]:
#grid options
grids = c('cru_spline_interp', 'era_mean')


In [6]:
#load gbd shapefile
gbd_shape = readOGR(paste0(j,'/DATA/SHAPE_FILES/GBD_geographies/master/GBD_2016/master/shapefiles'), 'GBD2016_analysis_final')

OGR data source with driver: ESRI Shapefile 
Source: "/home/j//DATA/SHAPE_FILES/GBD_geographies/master/GBD_2016/master/shapefiles", layer: "GBD2016_analysis_final"
with 747 features
It has 10 fields


In [22]:
#load list of cod locations
cod_locs = get_location_metadata(location_set_id = 35)

In [14]:
ll_locs = as.vector(cod_locs[is_estimate == 1,location_id])


In [18]:
output.dir = paste0(j,'/temp/dccasey/temperature/data/rasterized_shapefiles/')

In [20]:
for(ggg in grids){
    print(ggg)
    data.dir = paste0('/share/geospatial/temperature/estimates/',ggg,'/')
    temp_grid = brick(paste0(data.dir,list.files(path = data.dir, pattern = as.character(2000))))[[1]]
    
    #check for rotation
    rotate_me = ifelse(extent(temp_grid)[2] >183, T, F)
    if(rotate_me) temp_grid = rotate(temp_grid)
        
    #begin rasterizing
    prefix = substr(ggg, 1, 3)
    rasterize(gbd_shape[gbd_shape$level == 3,], temp_grid, 'loc_id', filename = paste0(output.dir,prefix,'admin0.tif'))
    rasterize(gbd_shape[gbd_shape$loc_id %in% ll_locs,], temp_grid, 'loc_id', filename = paste0(output.dir,prefix,'isestimate.tif'))
}

[1] "cru_spline_interp"
[1] "era_mean"
